<h1 align=center><font size = 5>Segmenting and Clustering Neigborhoods in Toronto</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href='#sec1'>Scrape Wikipedia to build the dataset</a>
    
2. [Get the latitude and longitude coordinates of a given postal code](#sec2)

3.  <a href="#sec3">Explore Neighborhoods in Toronto</a>
    
4.  <a href="#sec4"> Analyse the Neighborhoods</a>
5. <a href="#sec5"> Cluster the Neighborhoods</a>
        
    </font>
    </div>

Load all necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup # library to scape 

<a id='sec1'></a>

## 1. Scrape Wikipedia to build the dataset

#### Scrape the following wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M using BeautifulSoup and store the relevant information in the df_hoods dataframe.

In [2]:
# define the url to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# get the html data
html_doc  = requests.get(url).text

#parse the html document using beautiful soup
soup = BeautifulSoup(html_doc)

#scrape the table from the html document
table = soup.find('table')

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# create the dataframe
df_hoods = pd.DataFrame(columns=column_names)

#Get all rows from the table
iteritem = iter(table.find_all('tr')) # in html table row is represented by the tag <tr>
next(iteritem)
for row in iteritem: 
    # Get all columns in each row.
    cols = row.find_all('td') # in html a column is represented by the tag <td>
    postal_code = cols[0].getText().replace('\n','')
    borough = cols[1].getText().replace('\n','')
    neighborhood = cols[2].getText().replace('\n','')
    
    df_hoods = df_hoods.append({'PostalCode': postal_code,
                                'Borough': borough,
                                'Neighborhood': neighborhood}, ignore_index=True)
    
print('The data frame has {} rows'.format(df_hoods.shape[0]))
df_hoods.head()

The data frame has 180 rows


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Remove cells where a borough is not assigned

In [3]:
df_hoods = df_hoods.drop(df_hoods[df_hoods['Borough']=='Not assigned'].index).reset_index(drop=True)
print('There are {} remaining rows'.format(df_hoods.shape[0]))
df_hoods.head(11)

There are 103 remaining rows


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Check for neighborhoods with 'Not assigned' values but with assigned boroughs

In [4]:
temp = df_hoods[df_hoods['Neighborhood']=='Not assigned'].shape[0]
if temp==0:
    print('The are no \'Not assigned\' neighborhood entries.')
else:
    df_hoods['Neighborhood'][df_hoods['Neighborhood']=='Not assigned']=df_hoods['Borough']
    print('Not assigned neighborhoods were named after their borough. {} changes were made.'.format(temp))

The are no 'Not assigned' neighborhood entries.


In [5]:
print('The number of rows are', df_hoods.shape[0])

The number of rows are 103


<a id='sec2'></a>

## 2. Get the latitude and longitude coordinates of a given postal code

In [6]:
# load the geospatial data and display the head
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# create a set of postal codes from the two data frames and check if they are equivalent
setA = set(df_hoods['PostalCode'])
setB = set(df_coordinates['Postal Code'])
setA == setB

True

In [8]:
# append new columns in data frame
df_hoods['Latitude'] = np.nan
df_hoods['Longitude'] = np.nan

#loop through the coordinates data frame, get the respective row index in the neighborhood data frame and append the coordinates
for i in range(df_coordinates.shape[0]):
    row_id = df_hoods.index[df_hoods.loc[:,'PostalCode'] == df_coordinates.iloc[i,0]].tolist()
    df_hoods.iloc[row_id,3:5] = df_coordinates.iloc[i,1:3].tolist()


In [9]:
df_hoods.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<a id='sec3'></a>

## 3. Explore neighborhoods in Toronto

**Create a map of Toronto with neighborhoods superimposed on top.**

In [10]:
# get the coordinates for toronto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_hoods['Latitude'], df_hoods['Longitude'], df_hoods['Borough'], df_hoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto    

To simplify the analysis we shall work with only boroughs that contain the word Toronto. We start by finding these boroughs and creating a new data frame "df_toronto"

In [12]:
# find the boroughs that contain the word Toronto
df_toronto = df_hoods[df_hoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto.sort_values('Borough', inplace=True)
df_toronto = df_toronto.reset_index(drop=True)
temp = df_toronto['Borough'].unique()
filtered_boroughs = ', '.join(temp)
print('There exist {} boroughs that contain the word Toronto. '.format(len(temp)) + 'These are: ' +  filtered_boroughs + '.')
print('There exist {} neighborhoods within the {} boroughs.'.format(df_toronto.shape[0],len(temp)))

There exist 4 boroughs that contain the word Toronto. These are: Central Toronto, Downtown Toronto, East Toronto, West Toronto.
There exist 39 neighborhoods within the 4 boroughs.


In [13]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
3,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


Plot only  filtered neighborhoods

In [14]:
# create map of Toronto again using latitude and longitude from filtered boroughs
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto    

**Define Foursquare Credentials and Version**

In [15]:
client_id = '3VN3BW55EFHN2PZL3QCEGFNUXIEYKLNXUQ5PZV5J2NAC3SL5' # your Foursquare ID
client_secret = 'PKDIFCZGAY1QUPUKOTUQJAGHRUL23AVXPLLVBFMNSPE4CEY5' # your Foursquare Secret
version = '20180605' # Foursquare API version
limit = 100 # A default Foursquare API limit value

Lets explore the first neighborhood in our data frame, i.e., Roselawn

In [16]:
neighborhood_name = df_toronto.loc[0, 'Neighborhood']
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Roselawn are 43.7116948, -79.41693559999999.


In [17]:
#create the get request url
radius = 500
search_query = neighborhood_name
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(client_id,client_secret, neighborhood_latitude, neighborhood_longitude, version, radius, limit)

#send the get request
results = requests.get(url).json() 

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [19]:
venues = results['response']['groups'][0]['items'] 
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

nearby_venues.head()

3 venues were returned by Foursquare.


,name,categories,lat,lng
0,Ceiling Champions,Home Service,43.713891,-79.420702
1,Rosalind's Garden Oasis,Garden,43.712189,-79.411978
2,Menchie's St. Clair West,Ice Cream Shop,43.707664,-79.414301


**Repeat the analysis for all neighborhoods**

define a function that automates the previous steps and writes everything in "nearby_venues" data frame

In [20]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# define the inputs and call the function
input1 = df_toronto['Neighborhood'].values.tolist()
input2 = df_toronto['Latitude'].values.tolist()
input3 = df_toronto['Longitude'].values.tolist()


toronto_venues = getNearbyVenues(input1, input2, input3)

Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Lawrence Park
Kensington Market, Chinatown, Grange Park
Church and Wellesley
University of Toronto, Harbord
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Regent Park, Harbourfront
Commerce Court, Victoria Hotel
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
Toronto Dominion Centre, Design Exchange
St. James Town
Berczy Park
Central Bay Street
Harbourfront East, Union Station, Toronto Islands
Christie
Richmond, Adelaide, King
The Beaches
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
The Danforth West, Riverdale
India Bazaar, The Bea

Display the number of venues and the number of unique categories in the data frame

In [22]:
print('There are total {} venues.'.format(toronto_venues.shape[0]))
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are total 1630 venues.
There are 236 uniques categories.


In [23]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service
1,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
2,Roselawn,43.711695,-79.416936,Menchie's St. Clair West,43.707664,-79.414301,Ice Cream Shop
3,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


<a id='sec4'></a>

## 4. Analyse the neighborhoods

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1630, 237)


,Neighborhoood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhoood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(39, 237)


,Neighborhoood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.034483,0.000000,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.017241,0.017241,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.000000,0.017241,0.051724,0.103448,0.0,0.0,0.0,0.0,0.0,0.017241,0.000000,0.017241,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.000000,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.017241,0.0,0.0,0.0,0.0,0.000000,0.034483,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.01724

**Let's put the top 10 venues in each neighborhood in a new data frame**

First, let's write a function to sort the venues in descending order.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [27]:
num_top_venues = 10

indicators = ['st','nd','rd']
columns = ['Neighborhood']

for index in range(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(index+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(index+1))
        
# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns) 
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoood']

for index in range(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[index,1:] = return_most_common_venues(toronto_grouped.iloc[index,:], num_top_venues)
    
toronto_venues_sorted.head()

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Restaurant,Indian Restaurant,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant,Auto Workshop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Thai Restaurant,Bubble Tea Shop,Burger Joint,Ramen Restaurant,Indian Restaurant


<a id='sec5'></a>

## 5. Cluster neighborhoods

Run _k_-means to cluster the neighborhood into 4 clusters.

In [28]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhoood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [29]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
toronto_merged = df_toronto
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Garden,Ice Cream Shop,Home Service,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Gym / Fitness Center,Hotel,Park,Breakfast Spot,Food & Drink Shop,Sandwich Place,Department Store,Pizza Place,Convenience Store,Distribution Center
2,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
3,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Coffee Shop,Clothing Store,Yoga Studio,Ice Cream Shop,Salon / Barbershop,Fast Food Restaurant,Spa,Sporting Goods Shop,Diner,Restaurant
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,BBQ Joint,Pizza Place,Donut Shop,Burger Joint,Middle Eastern Restaurant,Pub,Indian Restaurant


Visualise the resulting clusters

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examine Clusters**

**Cluster 1**

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roselawn,Garden,Ice Cream Shop,Home Service,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio


**Cluster 2**

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Davisville North,Gym / Fitness Center,Hotel,Park,Breakfast Spot,Food & Drink Shop,Sandwich Place,Department Store,Pizza Place,Convenience Store,Distribution Center
2,"Forest Hill North & West, Forest Hill Road Park",Trail,Jewelry Store,Sushi Restaurant,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
3,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Yoga Studio,Ice Cream Shop,Salon / Barbershop,Fast Food Restaurant,Spa,Sporting Goods Shop,Diner,Restaurant
4,"The Annex, North Midtown, Yorkville",Sandwich Place,Café,Coffee Shop,BBQ Joint,Pizza Place,Donut Shop,Burger Joint,Middle Eastern Restaurant,Pub,Indian Restaurant
5,Davisville,Sandwich Place,Dessert Shop,Sushi Restaurant,Pizza Place,Café,Park,Thai Restaurant,Italian Restaurant,Gym,Coffee Shop
7,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Fried Chicken Joint,Pizza Place,Liquor Store,Restaurant,Bank,Bagel Shop,Supermarket,Sushi Restaurant,Pub
8,Lawrence Park,Park,Swim School,Bus Line,Business Service,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Dessert Shop,Arts & Crafts Store,Caribbean Restaurant,Bakery,Gaming Cafe
10,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Fast Food Restaurant,Men's Store,Hotel,Mediterranean Restaurant,Yoga Studio
11,"University of Toronto, Harbord",Café,Bakery,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bank,Beer Bar,Beer Store,Comfort Food Restaurant


**Cluster 3**

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Moore Park, Summerhill East",Park,Tennis Court,Trail,Restaurant,Yoga Studio,Dog Run,Dessert Shop,Diner,Discount Store,Distribution Center
13,Rosedale,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


**Cluster 4**

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,The Beaches,Health Food Store,Pub,Neighborhood,Trail,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center,Donut Shop
